In [113]:
import pandas as pd
from pathlib import Path
import os


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"

In [114]:
DATA_DIR = Path("data", "ijcnlp_dailydialog", "train")
data = pd.read_csv(Path(DATA_DIR, "dialogues_train.txt"),  delimiter = "\n", names = ["dialogues"])

In [116]:
def seputterances(row):
    try:
        row = row.split("__eou__")
        row = row[:-1]
        return row
    except:
        return row

data["dialogues"] = data["dialogues"].apply(seputterances)

data

,dialogues
0,"[Say , Jim , how about going for a few beers a..."
1,"[Can you do push-ups ? , Of course I can . It..."
2,"[Can you study with the radio on ? , No , I l..."
3,"[Are you all right ? , I will be all right so..."
4,"[Hey John , nice skates . Are they new ? , Ye..."
...,...
11113,"[Hello , I bought a pen in your shop just befo..."
11114,"[Do you have any seats available ? , Yes . Th..."
11115,"[Uncle Ben , how did the Forbidden City get th..."
11116,"[May I help you , sir ? , I want a pair of lo..."


In [117]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

In [118]:
dataset = {"S1":[], "S2": []}


for i in range(len(data)):
    for j in range(len(data["dialogues"][i])):
        dataset["S1"].append(data["dialogues"][i][j])
        try:
            dataset["S2"].append(data["dialogues"][i][j+1])
        except:
            dataset["S2"].append(tokenizer.eos_token)

In [119]:
tokenizer.pad_token = tokenizer.eos_token

tokenized_dataset = tokenizer(
    dataset["S1"],
    dataset["S2"],
    padding=True,
    truncation=True,
    return_tensors = "pt"
)

tokenized_dataset

{'input_ids': tensor([[25515,   837,  5395,  ..., 50256, 50256, 50256],
        [  921,   760,   326,  ..., 50256, 50256, 50256],
        [ 1867,   466,   345,  ..., 50256, 50256, 50256],
        ...,
        [  314,   761,  2546,  ..., 50256, 50256, 50256],
        [23722,   314,   423,  ..., 50256, 50256, 50256],
        [16805,   764,   314,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [120]:
import pickle 

with open(Path("data", "inturn_conversations.pkl"), 'wb') as handle:
    pickle.dump(tokenized_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [121]:
with open(Path("data", "inturn_conversations.pkl"), 'rb') as handle:
    tokenized_dataset = pickle.load(handle)

In [123]:
import os.path
os.path.isfile(Path("data", "inturn_conversations.pkl")) 

True

In [105]:
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

In [108]:
import torch
from transformers import AdamW
from transformers import get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3

num_batch = 2

num_training_steps = int(num_epochs * len(tokenized_dataset["input_ids"])/4)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

print(num_training_steps, device)

77235 cuda


In [109]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):
    for i in range(len(tokenized_dataset["input_ids"])):
        inputs= {}

        ids = tokenized_dataset["input_ids"][:num_batch]
        mask = tokenized_dataset["attention_mask"][:num_batch]

        inputs["input_ids"] = ids.to(device)
        inputs["attention_mask"] = mask.to(device)

        outputs = model(**inputs, labels = inputs["input_ids"])
        print(outputs)

        loss = outputs.loss
        
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        
        progress_bar.update(1)
        
        break


  0%|          | 0/77235 [00:10<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 15.90 GiB total capacity; 4.54 GiB already allocated; 23.75 MiB free; 4.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF